# Practica dos

Grupo 14:
* Joaquín Ibáñez Penalva
* Aurora Zuoris

Para la realización de esta práctica  se usará la librería de numpy y sklearn.

In [2]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split

In [7]:
df = pd.read_csv('tiempos.csv', sep=';')
df.head()

,P1,P2,P3,P4,P5,P6,T_Final
0,02:52:01,04:14:12,05:56:53,06:25:24,08:34:23,10:36:25,11:43:18
1,02:43:32,04:13:40,06:11:44,06:48:12,09:16:50,11:14:02,12:18:22
2,03:08:03,04:39:34,06:29:37,06:59:38,09:40:45,12:08:35,13:23:45
3,03:15:20,04:48:13,06:37:29,07:05:43,09:39:46,12:09:50,13:20:18
4,03:15:53,04:56:32,06:55:22,07:33:08,10:19:46,12:46:40,14:23:46


In [9]:
def hhmmss_to_mins(x):
	h, m, s = map(int, x.split(':'))
	return h * 60 + m + s / 60

df = df.applymap(hhmmss_to_mins)
df.head()

AttributeError: 'float' object has no attribute 'split'

In [ ]:
data = df.to_numpy

## Ejercicio 1

## Ejercicio 2